#### the glob package needed to install you can use the following command

In [11]:
!pip install glob2

  Created wheel for glob2: filename=glob2-0.7-py2.py3-none-any.whl size=9320 sha256=14cd24eb2a6c27556c188bbb32eb0c3a50da6af9d8f839b4c434ec65f55bbd5d
  Stored in directory: c:\users\ahmed saad\appdata\local\pip\cache\wheels\96\2f\ad\5bba2807c8e762a019e4abd6f70dafbab1889acda23192aeff
Successfully built glob2


#### Import all the needed packages 

In [1]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import glob

#### Function to detect all the edges in the test videos and imgs by using the canny method

In [2]:
def canny(img):
    canny = cv.Canny(img, 50, 150)
    return canny

#### Function to determine the region of interest to fill it by white and mask it with the original image when we convert the all of the pixels of the image to zero then it will return the image with the region of interest in white and all other pixels are black

In [3]:
def region_of_interest(img):
    height = img.shape[0]
    poly = np.array([[(200,height), (1200, height), (790,450), (580, 450)]])
    mask = np.zeros_like(img)
    cv.fillPoly(mask, poly, 255)
    return mask

#### Function to loop over the lines in the imgs or videos and convert it to 1D and draw the lines on the img with a certain color and thickness

In [30]:
def display_lines(img, lines):
    line_image = np.zeros_like(img)
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line.reshape(4)
            cv.line(line_image, (x1,y1), (x2,y2), (255,0,0), 10)
    return line_image

#### Function that loop over all the lines comes from the hough transform to know the slope and the intercept of each line and by checking we notice that the left lines have a negative slope and the right a postitive slope so we made arrays to put the slopes,intercepts of left lines in an array and another one for the right lines and then we add more weight to the longer lines (calculate average) by knowing the length of each line and make a dot multiplication with the array which contain the slope and intercept of each line divide by the sum of the length of each line and finally return the slope and intercept of left lane and right lane 

In [5]:
def average_slope_intercept(lines):
    left_lines    = [] # (slope, intercept)
    left_weights  = [] # (length,)
    right_lines   = [] # (slope, intercept)
    right_weights = [] # (length,)
    
    for line in lines:
        for x1, y1, x2, y2 in line:
            if x2==x1:
                continue # ignore a vertical line
            slope = (y2-y1)/(x2-x1)
            intercept = y1 - slope*x1
            length = np.sqrt((y2-y1)**2+(x2-x1)**2)
            if slope < 0: # y is reversed in image
                left_lines.append((slope, intercept))
                left_weights.append((length))
            else:
                right_lines.append((slope, intercept))
                right_weights.append((length))
    
    # add more weight to longer lines    
    left_lane  = np.dot(left_weights,  left_lines) /np.sum(left_weights)  if len(left_weights) >0 else None
    right_lane = np.dot(right_weights, right_lines)/np.sum(right_weights) if len(right_weights)>0 else None
    
    return left_lane, right_lane # (slope, intercept), (slope, intercept)

#### Function to convert the left and right lines that represnted in slope and intercept to pixel points and make sure that the coordinates of the line are integer because the cv.line requires it and then return the line according to the calling of the fucntion by the arugment of the left line or the right line

In [6]:
def make_line_points(y1, y2, line,old_slope):
    if line is None:
        return "not"
    
    slope, intercept = line
    #print("slope = ", slope)
    #print("intercept = ", intercept)
    
    # make sure everything is integer as cv2.line requires it
    x1=int((y1 - intercept)/old_slope) if slope ==0 else int((y1 - intercept)/slope)
    x2= int((y2 - intercept)/old_slope) if slope ==0 else int((y2 - intercept)/slope)
    y1 = int(y1)
    y2 = int(y2)
    
    return ((x1, y1), (x2, y2))

##### Function that calls the average_slope_intercept  function that return the line represnted in slope and intercept and then call make_line_points twice to convert the lines that represnted in slope and intercept to pixel points to the right and left lanes and return them and the draw_lane_lines function is a function to draw the lane lines on the original image by looping over every line and draw it on the original image after convert it to black image as cv.line require that then add the image that has the lines on it to the original image so the result we draw lines on the lanes in the original image

In [7]:
def lane_lines(image, lines,left_slope,right_slope):
    left_lane, right_lane = average_slope_intercept(lines)
    
    y1 = 680 # bottom of the image
    y2 = y1*0.725     # slightly lower than the middle

    left_line  = make_line_points(y1, y2, left_lane,left_slope)
    right_line = make_line_points(y1-5, y2, right_lane,right_slope)
    return left_line, right_line,left_lane,right_lane

    
def draw_lane_lines(image, lines, color=[255,0, 0], thickness=10):
    # make a separate image to draw lines and combine with the orignal later
    line_image = np.zeros_like(image)
    for line in lines:
        if line != "not":
            cv.line(line_image, *line,  color, thickness)
    # image1 * α + image2 * β + λ
    # image1 and image2 must be the same shape.
    return cv.addWeighted(image, 1.0, line_image, 0.8, 0.0)

#### Function that mask the image after converting it to black with the region of interest by know the coordinates of the left and right line

In [8]:
def lane(left_line,right_line,img):
    poly = np.array([[(left_line[0][0],left_line[0][1]), (right_line[0][0], right_line[0][1]),  (right_line[1][0], right_line[1][1]),(left_line[1][0],left_line[1][1])]])
    mask = np.zeros_like(img)
    cv.fillPoly(mask, poly, color=(0, 255, 0))
    return mask

## Detect lanes in the videos
1) Read the video and covert every frame in it to hls \
2) We convert to able us to determine the yellow color of the lines \
3) To reduce the noise we will apply gaussianBlur after convert the every frame to grayscale \
4) We will call the reigon of interest function and make and make AND operation with the blur video to detect the region \
5) We will make the binary threshold to the blur video to detect the white lines \
6) We will make OR operation between the mask video that detect yellow lines and the video with the threshold that detect the white lines and then We will make AND operation between the result video from the OR and the result from the region of interest function if we pass it the video that result from the OR to detect the lines in the region of interest \
7) Then apply hough transform in the video that has the lines detected in the region of interest and pass the result to the function lane lines that return the left and right lanes \
8) We also handle the blanking of the lines \
9) Then we call the lane and draw lane lines functions to combine the results of them into one video and adding weights to each one so after this step the original video has lines on the white and yellow lanes and fill the region of interest between them \
10) Determine the center of the lane by mapping of the actual distance in meters to pixel by knowing the center of the lane and the car\
11) Debugging mode also by put with the output videos ( the hough result ,Canny result,Region of interest and The filling between the lanes )


In [50]:
cap = cv.VideoCapture('project_video.mp4')
# Check if camera opened successfully
if (cap.isOpened()== False): 
  print("Error opening video stream or file")

left_line_old =((200,680),(790,450))
right_line_old=((1200,680),(580,450))
left_slope_old =100
right_slope_old=100

while(cap.isOpened()):
  # Capture frame-by-frame
  ret, frame = cap.read()
  if ret == True:
    # Display the resulting frame
  
    hls = cv.cvtColor(frame, cv.COLOR_BGR2HLS)    
    low_yellow = np.uint8([ 10,   0, 100])
    up_yellow = np.uint8([ 40, 255, 255])
    mask = cv.inRange(hls, low_yellow, up_yellow)
    gray = cv.cvtColor(frame, cv.COLOR_RGB2GRAY)
    blur = cv.GaussianBlur(gray, (3,3), 0)
    interestt = blur & region_of_interest(blur)
    
    ret, thresh = cv.threshold(interestt, 210, 240, cv.THRESH_BINARY)
    
    new = mask | thresh
    
    interestt = new & region_of_interest(new)

    new_canny = cv.Canny(interestt, 50, 150)
    lines = cv.HoughLinesP(new_canny, 5, np.pi/180, 100, np.array([]), minLineLength=50, maxLineGap=200)
    line_img = display_lines(frame, lines)
    
    left_line, right_line , left_slope, right_slope=lane_lines(frame, lines,left_slope_old,right_slope_old) 
    new_lines=left_line, right_line
    
    new_lines=list(new_lines)
    if  new_lines[0] == "not":
         new_lines[0]=left_line_old
    else:
        left_line_old= new_lines[0]

        
    if new_lines[1] == "not":
        new_lines[1]=right_line_old
    else:
        right_line_old=new_lines[1]
    
    if left_slope is not None:
        if left_slope[0] != 0:
            left_slope_old=left_slope[0]
    if right_slope is not None:
        if right_slope[0] !=0:
            right_slope_old=right_slope[0]
        
    
    lane_mask = lane( new_lines[0],new_lines[1],frame)

    x = draw_lane_lines(frame,new_lines)    
    lanes_img = cv.addWeighted(x, 1.0, lane_mask, 1.0, 0.0)
    
    
    lane_centre= ((new_lines[1][0][0]-new_lines[0][0][0])/2)+new_lines[0][0][0]
    width=frame.shape[1]
    car_centre=(width)/2
    diffrence=lane_centre-car_centre
    meters=diffrence*3/(new_lines[1][0][0]-new_lines[0][0][0])
    
    font = cv.FONT_HERSHEY_SIMPLEX
    meter =round(meters, 3)
    if meters >0:
        cv.putText(lanes_img,'Vehicle is '+ str(meter)+'m left of centre',(50, 50),font, 1, (255, 255, 255), 2, cv.LINE_4)
    elif meters<0:
        cv.putText(lanes_img,'Vehicle is '+ str(meter*-1)+'m right of centre',(50, 50),font, 1, (255, 255, 255), 2, cv.LINE_4)
    else: 
        cv.putText(lanes_img,'Vehicle is at centre',(50, 50),font, 1, (255, 255, 255), 2, cv.LINE_4)

    dim = (320,200)
    resized_interest= cv.resize(new_canny, dim)
    resized_thresh= cv.resize(interestt, dim)
    resized_mask= cv.resize(line_img, dim)
    resized_primary = cv.resize(lane_mask, dim)

    # Convert grayscale image to 3-channel image,so that they can be stacked together    
    resized_interest = cv.cvtColor(resized_interest,cv.COLOR_GRAY2BGR)
    resized_thresh= cv.cvtColor(resized_thresh,cv.COLOR_GRAY2BGR)
    #resized_primary = cv.cvtColor(resized_primary, cv.COLOR_GRAY2BGR)

    resized_interest = resized_interest[60:320, :]
    resized_thresh = resized_thresh[60:320, :]
    resized_primary = resized_primary[60:320, :]
    resized_mask = resized_mask[60:320, :]
    
    cv.putText(resized_interest,'Canny Edge Detection',(20, 30),font, 0.7, (255, 255, 255), 1, cv.LINE_4)
    cv.putText(resized_thresh,'Region of Interest',(20, 30),font, 0.7, (255, 255, 255), 1, cv.LINE_4)
    cv.putText(resized_primary,'Lane Fill',(20, 30),font, 0.7, (255, 255, 255), 1, cv.LINE_4)
    cv.putText(resized_mask,'Hough Lines',(20, 30),font, 0.7, (255, 255, 255), 1, cv.LINE_4)

    
    hori = np.concatenate((resized_interest, resized_thresh), axis = 1)
    hori = np.concatenate((hori, resized_mask), axis = 1)
    hori = np.concatenate((hori, resized_primary), axis = 1)
    vert = np.concatenate((lanes_img, hori), axis= 0)

    #resized_primary = cv.resize(vert, (1920,720))
    cv.imshow('Frame',vert)
       
    # Press Q on keyboard to  exit
    if cv.waitKey(25) & 0xFF == ord('q'):
      break

  else: 
    break

# When everything done, release the video capture object
cap.release()

# Closes all the frames
cv.destroyAllWindows()

## Detect lanes in the test images
1) Read the img and covert in it to hsv \
2) We convert to able us to determine the yellow color of the lines \
3) To reduce the noise we will apply gaussianBlur after convert the image to grayscale \
4) We will call the reigon of interest function and make and make AND operation with the blur img to detect the region \
5) We will make the binary threshold to the blur img to detect the white lines \
6) We will make OR operation between the mask img that detect yellow lines and the img with the threshold that detect the white lines and then We will make AND operation between the result img from the OR and the result from the region of interest function if we pass it the img that result from the OR to detect the lines in the region of interest \
7) Then apply hough transform in the img that has the lines detected in the region of interest and pass the result to the function lane lines that return the left and right lanes \
8) Then we call the lane and draw lane lines functions to combine the results of them into one img and adding weights to each one so after this step the original img has lines on the white and yellow lanes and fill the region of interest between them 
9) Determine the center of the lane by mapping of the actual distance in meters to pixel by knowing the center of the lane and the car\
10) Debugging mode also by put with the output of the test imgs( the hough result ,Canny result,Region of interest and The filling between the lanes )


In [51]:
# Reading an image
images_paths = glob.glob("images/*.jpg")
images = [cv.imread(image) for image in images_paths]

# getting img dimensions and printing it
image_idx = 0

#dummy
left_line_old =((200,680),(790,450))
right_line_old=((1200,680),(580,450))
left_slope_old =100
right_slope_old=100

for test_img in images:
    hls = cv.cvtColor(test_img, cv.COLOR_BGR2HLS)    
    low_yellow = np.uint8([ 10,   0, 100])
    up_yellow = np.uint8([ 40, 255, 255])
    mask = cv.inRange(hls, low_yellow, up_yellow)
    gray_img = cv.cvtColor(test_img, cv.COLOR_RGB2GRAY)
    blur_img = cv.GaussianBlur(gray_img, (3,3), 0)    
    blur_interest_img = blur_img & region_of_interest(blur_img)
    ret, thresh = cv.threshold(blur_interest_img, 210, 255, cv.THRESH_BINARY)
    two_lanes = mask | thresh
    interest_img = two_lanes & region_of_interest(two_lanes)
    new_canny = cv.Canny(interest_img, 50, 150)
    lines = cv.HoughLinesP(new_canny, 5, np.pi/180, 100, np.array([]), maxLineGap=300)
    line_img = display_lines(test_img, lines)
    left_line, right_line , left_slope, right_slope=lane_lines(test_img, lines,left_slope_old,right_slope_old) 
    new_lines=left_line, right_line
    new_lines=list(new_lines)
    if  new_lines[0] == "not":
         new_lines[0]=left_line_old
    else:
        left_line_old= new_lines[0]
      
    if new_lines[1] == "not":
        new_lines[1]=right_line_old
    else:
        right_line_old=new_lines[1]
    
    lane_centre= ((new_lines[1][0][0]-new_lines[0][0][0])/2)+new_lines[0][0][0]
    width=test_img.shape[1]
    car_centre=(width)/2
    diffrence=lane_centre-car_centre
    meters=diffrence*3/(new_lines[1][0][0]-new_lines[0][0][0])
    mask_lanes = lane(new_lines[0],new_lines[1],test_img)
    x = draw_lane_lines(test_img,new_lines)    
    lanes_img = cv.addWeighted(x, 1.0, mask_lanes, 0.8, 0.0)
    font = cv.FONT_HERSHEY_SIMPLEX
    meter =round(meters, 2)
    if meters >0:
        cv.putText(lanes_img,'Vehicle is '+ str(meter)+'m left of centre',(50, 50),font, 1, (255, 255, 255), 2, cv.LINE_4)
    elif meters<0:
        cv.putText(lanes_img,'Vehicle is '+ str(meter*-1)+'m right of centre',(50, 50),font, 1, (255, 255, 255), 2, cv.LINE_4)
    else:
        cv.putText(lanes_img,'Vehicle is at centre',(50, 50),font, 1, (255, 255, 255), 2, cv.LINE_4)
   
    
    dim = (320,200)
    resized_interest= cv.resize(new_canny, dim)
    resized_thresh= cv.resize(interest_img, dim)
    resized_mask= cv.resize(line_img, dim)
    resized_primary = cv.resize(mask_lanes, dim)

    # Convert grayscale image to 3-channel image,so that they can be stacked together    
    resized_interest = cv.cvtColor(resized_interest,cv.COLOR_GRAY2BGR)
    resized_thresh= cv.cvtColor(resized_thresh,cv.COLOR_GRAY2BGR)
    #resized_primary = cv.cvtColor(resized_primary, cv.COLOR_GRAY2BGR)

    resized_interest = resized_interest[60:320, :]
    resized_thresh = resized_thresh[60:320, :]
    resized_primary = resized_primary[60:320, :]
    resized_mask = resized_mask[60:320, :]
    
    cv.putText(resized_interest,'Canny Edge Detection',(20, 30),font, 0.7, (255, 255, 255), 1, cv.LINE_4)
    cv.putText(resized_thresh,'Region of Interest',(20, 30),font, 0.7, (255, 255, 255), 1, cv.LINE_4)
    cv.putText(resized_primary,'Lane Fill',(20, 30),font, 0.7, (255, 255, 255), 1, cv.LINE_4)
    cv.putText(resized_mask,'Hough Lines',(20, 30),font, 0.7, (255, 255, 255), 1, cv.LINE_4)

    
    hori = np.concatenate((resized_interest, resized_thresh), axis = 1)
    hori = np.concatenate((hori, resized_mask), axis = 1)
    hori = np.concatenate((hori, resized_primary), axis = 1)
    vert = np.concatenate((lanes_img, hori), axis= 0)
    
    cv.imwrite('output/test{}.jpg'.format(str(image_idx)), vert)
    image_idx = image_idx + 1